In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Validation set (5000, 28, 28) (5000,)
Test set (5000, 28, 28) (5000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28, 1) (100000, 10)
Validation set (5000, 28, 28, 1) (5000, 10)
Test set (5000, 28, 28, 1) (5000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [61]:
batch_size = 16
patch_size = 7
patch_size2 = 5
depth = 16
num_hidden = 64
keepProb = 1.0;
inception_depth = 8

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size2, patch_size2, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  #oneByOneWeights = tf.Variable(tf.truncated_normal(
  #    [1, 1, depth, depth], stddev=0.2))
  #oneByOneBias = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
  fully_connected_layer_weights = tf.Variable(tf.truncated_normal(
  [image_size * image_size * (3 * inception_depth + 1), num_hidden], stddev=0.1))
  fully_connected_layer_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  def inception2d(x, in_channels, filter_count):
      # bias dimension = 3*filter_count and then the extra in_channels for the avg pooling
      outDim = 3*filter_count + in_channels
      bias = tf.Variable(tf.truncated_normal([outDim], stddev=0.01)),
    
      # 1x1
      one_filter = tf.Variable(tf.truncated_normal([1, 1, in_channels, filter_count], stddev=0.01))
      one_by_one = tf.nn.conv2d(x, one_filter, strides=[1, 1, 1, 1], padding='SAME')
    
      # 3x3
      three_filter = tf.Variable(tf.truncated_normal([3, 3, in_channels, filter_count], stddev=0.01))
      three_by_three = tf.nn.conv2d(x, three_filter, strides=[1, 1, 1, 1], padding='SAME')

      # 5x5
      five_filter = tf.Variable(tf.truncated_normal([5, 5, in_channels, filter_count], stddev=0.01))
      five_by_five = tf.nn.conv2d(x, five_filter, strides=[1, 1, 1, 1], padding='SAME')

      # avg pooling
      pooling = tf.nn.avg_pool(x, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME')

      activations = tf.concat([one_by_one, three_by_three, five_by_five, pooling], axis=3)  # Concat in the 4th dim to stack
      logit = activations + bias
    
      
      return tf.nn.relu(logit)
  
    # Model.
  def model(data, bDropout):
    #conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    #conv = tf.nn.max_pool(data, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + layer1_biases)
    hidden = inception2d(data, num_channels, inception_depth)
    if bDropout:
        hidden = tf.nn.dropout(hidden, keep_prob=keepProb)
    #conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    #conv = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + layer2_biases)
    #if bDropout:
    #    hidden = tf.nn.dropout(hidden, keep_prob=keepProb)
    #conv = tf.nn.conv2d(hidden, oneByOneWeights, [1, 1, 1, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + oneByOneBias)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, fully_connected_layer_weights) + fully_connected_layer_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(model(tf_train_dataset, False))
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, False))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, False))


In [62]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.792863
Minibatch accuracy: 6.2%
Validation accuracy: 13.8%
Minibatch loss at step 50: 1.277547
Minibatch accuracy: 37.5%
Validation accuracy: 35.6%
Minibatch loss at step 100: 0.620330
Minibatch accuracy: 43.8%
Validation accuracy: 46.0%
Minibatch loss at step 150: 0.678696
Minibatch accuracy: 56.2%
Validation accuracy: 44.3%
Minibatch loss at step 200: 0.939497
Minibatch accuracy: 56.2%
Validation accuracy: 52.1%
Minibatch loss at step 250: 0.429354
Minibatch accuracy: 43.8%
Validation accuracy: 37.6%
Minibatch loss at step 300: 1.366650
Minibatch accuracy: 37.5%
Validation accuracy: 40.6%
Minibatch loss at step 350: 0.450921
Minibatch accuracy: 37.5%
Validation accuracy: 45.0%
Minibatch loss at step 400: 0.859467
Minibatch accuracy: 56.2%
Validation accuracy: 48.1%
Minibatch loss at step 450: 0.752243
Minibatch accuracy: 37.5%
Validation accuracy: 42.8%
Minibatch loss at step 500: 0.572363
Minibatch accuracy: 50.0%
Validation accuracy: 39.3%
Mi